In [0]:
# https://keras.io/
!pip install -q keras
import keras

In [0]:
import keras
from keras.datasets import cifar10
from keras.models import Model, Sequential
from keras.layers import Dense, Dropout, Flatten, Input, AveragePooling2D, merge, Activation
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.layers import Concatenate
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint
from keras.callbacks import EarlyStopping,ReduceLROnPlateau

In [0]:
# this part will prevent tensorflow to allocate all the avaliable GPU Memory
# backend
import tensorflow as tf
from keras import backend as k

# Don't pre-allocate memory; allocate as-needed
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

# Create a session with the above options specified.
k.tensorflow_backend.set_session(tf.Session(config=config))

In [0]:
# Hyperparameters
batch_size = 128
num_classes = 10
epochs = 50
l = 12
num_filter = 64
compression = 0.5
dropout_rate = 0.3

In [5]:
# Load CIFAR10 Data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
img_height, img_width, channel = x_train.shape[1],x_train.shape[2],x_train.shape[3]

# convert to one hot encoing 
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

170500096/170498071 [==============================] - 23s 0us/step


In [0]:
# Dense Block
def add_denseblock(input, num_filter = 72, dropout_rate = 0.25):
    global compression
    temp = input
    for _ in range(l):
        BatchNorm = BatchNormalization()(temp)
        relu = Activation('relu')(BatchNorm)
        Conv2D_3_3 = Conv2D(int(num_filter*compression), (3,3), use_bias=False ,padding='same')(relu)
        if dropout_rate>0:
          Conv2D_3_3 = Dropout(dropout_rate)(Conv2D_3_3)
        concat = Concatenate(axis=-1)([temp,Conv2D_3_3])
        
        temp = concat
        
    return temp

In [0]:
def add_transition(input, num_filter = 64, dropout_rate = 0.3):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    Conv2D_BottleNeck = Conv2D(int(num_filter*compression), (1,1), use_bias=False ,padding='same')(relu)
    if dropout_rate>0:
      Conv2D_BottleNeck = Dropout(dropout_rate)(Conv2D_BottleNeck)
    avg = AveragePooling2D(pool_size=(2,2))(Conv2D_BottleNeck)
    
    return avg

In [0]:
def output_layer(input):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    AvgPooling = AveragePooling2D(pool_size=(2,2))(relu)
    flat = Flatten()(AvgPooling)
    output = Dense(num_classes, activation='softmax')(flat)
    
    return output

In [0]:
num_filter = 64
dropout_rate = 0.3
l = 12
input = Input(shape=(img_height, img_width, channel,))
First_Conv2D = Conv2D(num_filter, (3,3), use_bias=False ,padding='same')(input)

First_Block = add_denseblock(First_Conv2D, num_filter, dropout_rate)
First_Transition = add_transition(First_Block, num_filter, dropout_rate)

Second_Block = add_denseblock(First_Transition, num_filter, dropout_rate)
Second_Transition = add_transition(Second_Block, num_filter, dropout_rate)

Third_Block = add_denseblock(Second_Transition, num_filter, dropout_rate)
Third_Transition = add_transition(Third_Block, num_filter, dropout_rate)

Last_Block = add_denseblock(Third_Transition,  num_filter, dropout_rate)
output = output_layer(Last_Block)


In [0]:
filepath="weights.best.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')

reduce_lr = ReduceLROnPlateau(monitor='val_loss', 
                              factor=0.1, 
                              patience=3, 
                              mode='auto', 
                              min_delta=0.0001, 
                              cooldown=0, 
                              min_lr=0)

callbacks_lists = [reduce_lr,checkpoint]

In [24]:
model = Model(inputs=[input], outputs=[output])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 32, 32, 64)   1728        input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 32, 32, 64)   256         conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 32, 32, 64)   0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
conv2d_2 (

In [0]:
# determine Loss function and Optimizer
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])

In [0]:
model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    callbacks=callbacks_lists,
                    validation_data=(x_test, y_test))

Train on 50000 samples, validate on 10000 samples
Epoch 1/50
50000/50000 [==============================] - 488s 10ms/step - loss: 1.2643 - acc: 0.5382 - val_loss: 1.9069 - val_acc: 0.4960

Epoch 00001: val_acc improved from -inf to 0.49600, saving model to weights.best.hdf5
Epoch 2/50
14464/50000 [=======>......................] - ETA: 5:09 - loss: 0.9905 - acc: 0.6484

50000/50000 [==============================] - 471s 9ms/step - loss: 0.9241 - acc: 0.6699 - val_loss: 0.9348 - val_acc: 0.6901

Epoch 00002: val_acc improved from 0.49600 to 0.69010, saving model to weights.best.hdf5
Epoch 3/50
30976/50000 [=================>............] - ETA: 2:45 - loss: 0.7797 - acc: 0.7255

50000/50000 [==============================] - 470s 9ms/step - loss: 0.7584 - acc: 0.7332 - val_loss: 1.2980 - val_acc: 0.6256

Epoch 00003: val_acc did not improve from 0.69010
Epoch 4/50
40576/50000 [=======================>......] - ETA: 1:22 - loss: 0.6554 - acc: 0.7713

50000/50000 [==============================] - 471s 9ms/step - loss: 0.6547 - acc: 0.7714 - val_loss: 1.1145 - val_acc: 0.6582

Epoch 00004: val_acc did not improve from 0.69010
Epoch 5/50
43648/50000 [=========================>....] - ETA: 55s - loss: 0.5807 - acc: 0.7963

50000/50000 [==============================] - 470s 9ms/step - loss: 0.5832 - acc: 0.7953 - val_loss: 0.6945 - val_acc: 0.7824

Epoch 00005: val_acc improved from 0.69010 to 0.78240, saving model to weights.best.hdf5
Epoch 6/50
38912/50000 [======================>.......] - ETA: 1:36 - loss: 0.5326 - acc: 0.8123

...

50000/50000 [==============================] - 471s 9ms/step - loss: 0.2488 - acc: 0.9130 - val_loss: 0.6102 - val_acc: 0.8414

Epoch 00018: val_acc did not improve from 0.84150
Epoch 19/50
44160/50000 [=========================>....] - ETA: 50s - loss: 0.2466 - acc: 0.9146

50000/50000 [==============================] - 470s 9ms/step - loss: 0.2462 - acc: 0.9149 - val_loss: 0.6100 - val_acc: 0.8409

Epoch 00019: val_acc did not improve from 0.84150
Epoch 20/50
44544/50000 [=========================>....] - ETA: 47s - loss: 0.2490 - acc: 0.9138

50000/50000 [==============================] - 471s 9ms/step - loss: 0.2476 - acc: 0.9143 - val_loss: 0.6047 - val_acc: 0.8427

Epoch 00020: val_acc improved from 0.84150 to 0.84270, saving model to weights.best.hdf5
Epoch 21/50
39040/50000 [======================>.......] - ETA: 1:35 - loss: 0.2472 - acc: 0.9144

50000/50000 [==============================] - 471s 9ms/step - loss: 0.2472 - acc: 0.9142 - val_loss: 0.6067 - val_acc: 0.8418

Epoch 00021: val_acc did not improve from 0.84270
Epoch 22/50
43008/50000 [========================>.....] - ETA: 1:00 - loss: 0.2449 - acc: 0.9146

50000/50000 [==============================] - 471s 9ms/step - loss: 0.2468 - acc: 0.9139 - val_loss: 0.6059 - val_acc: 0.8424

Epoch 00022: val_acc did not improve from 0.84270
Epoch 23/50
30080/50000 [=================>............] - ETA: 2:53 - loss: 0.2490 - acc: 0.9131

In [0]:
# Test the model
score = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

10000/10000 [==============================] - 13s 1ms/step
Test loss: 0.7949684534072876
Test accuracy: 0.7609


In [0]:
# Save the trained weights in to .h5 format
model.save_weights("DNST_model.h5")
print("Saved model to disk")

Saved model to disk


In [0]:
from google.colab import files

files.download('DNST_model.h5')